In [ ]:
from functools import partial, reduce
from itertools import accumulate, combinations, permutations, product
from operator import gt, or_

import numpy as np

In [ ]:
def overtone_to_semitones(n):
    overtones = np.arange(n) + 1
    steps = (12 * np.log2(overtones)) % 12
    return steps

In [ ]:
def distance_to_nearest_integer(ts):
    nint = np.round(ts)
    return np.minimum(abs(ts - nint), abs(nint - ts))

In [ ]:
distance_to_nearest_integer(overtone_to_semitones(16))

In [ ]:
print(overtone_to_semitones(16))
print(overtone_to_semitones(16) + 0.21)
# 4, 10, 6, 11

In [ ]:
base = overtone_to_semitones(16)
altered = None
best = 0
bestest = None
n_better = 0

for delta in range(1, 51):
    shifted = base + delta / 100

    # get indices where shifted is a better approximation than standard
    loc = distance_to_nearest_integer(shifted) < distance_to_nearest_integer(base)
    new_n_better = sum(loc)
    if new_n_better > n_better:
        n_better = new_n_better
        altered = shifted
        bestest = loc
        best = delta

print(best, sum(bestest))
print("============")
for i, (s, a, w) in enumerate(zip(base, altered, bestest)):
    print(
        i + 1,
        "\t",
        np.round(s),
        "\t",
        np.round(a),
        "\t",
        ["-", "*"][w.astype("int")],
        f"{distance_to_nearest_integer(a):.2f}",
    )

In [ ]:
def transpose(x, n):
    return x[-n:] + x[:-n]


class Necklace(tuple):
    def __eq__(self, other):
        for idx in range(len(self)):
            if tuple.__eq__(transpose(self, idx), other):
                return True
        return False

In [ ]:
def join(a, b):
    assert a[-1] == b[0]
    return a[:-1] + b

In [ ]:
def is_proper(scale):
    steps = len(scale)

    compounds = []
    for group in range(1, steps):
        compounds.append([sum(scale[(i + g) % steps] for g in range(group)) for i in range(steps)])

    return all(all(a < b for a, b in product(c1, c2)) for c1, c2 in zip(compounds, compounds[1:]))

In [ ]:
def get_harmonic_scales(seconds, thirds, edo=31):
    scales = []
    pairs = [list(p) for p in product(seconds, seconds) if sum(p) in thirds]
    matching_pairs = [join(*s) for s in product(pairs, pairs) if s[0][-1] == s[1][0]]

    # what is this 6?
    for _ in range(9):
        matching_pairs = [join(*s) for s in product(matching_pairs, pairs) if s[0][-1] == s[1][0]]
        matching_pairs = [s for s in matching_pairs if sum(s) <= edo]
        for s in matching_pairs:
            if sum(s) == edo and [s[-1], s[0]] in pairs and is_proper(s):
                if not any([Necklace(s) == Necklace(d) for d in scales]):
                    scales.append(s)
        matching_pairs = [s for s in matching_pairs if sum(s) < edo]

    return scales

In [ ]:
def delta_to_cents(deltas, edo=31):
    return list(accumulate([(12 / edo) * d for d in deltas]))

In [ ]:
def count_n_5ths(scale, fifth_step=18):
    n5ths = 0
    for i in range(len(scale)):
        if sum(transpose(scale, i)[:4]) == fifth_step:
            n5ths += 1

    return n5ths

In [ ]:
# Only for 31-EDO


def distance_from_major(scale):
    major = tuple(accumulate([3, 5, 5, 3, 5, 5, 5], initial=0))
    distance = 99999
    for i in range(len(scale)):
        ts = transpose(scale, i)
        ts = tuple(accumulate(ts, initial=0))
        d = sum([abs(x - y) for x, y in zip(major, ts)])
        if d < distance:
            distance = d

    return distance

In [ ]:
allowed_seconds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
allowed_thirds = [7, 8, 9, 10, 11]
scales = get_harmonic_scales(allowed_seconds, allowed_thirds, edo=31)
scales.sort(key=count_n_5ths, reverse=True)

In [ ]:
print(len(scales))
for s in scales:
    print(
        str(count_n_5ths(s))
        + ": "
        + str(s)
        + " => "
        + str([f"{p:.2f}" for p in delta_to_cents(s)])
    )

In [ ]:
for i in [5, 6, 7, 8, 9]:
    diatonic = [s for s in scales if len(s) == i]
    print(i, len(diatonic))

In [ ]:
[s for s in scales if set(s) == set([3, 5, 7])]

In [ ]:
[s for s in scales if 7 in s and len(s) == 7]

# Notes

subminor and supermajor 3rds come from the 7th harmonic
neutral 3rd comes from the 11th harmonic

31-EDO captures neutral 3rds better than sub/super minor/major 3rds

# standard triadic modes of new ottoman
```
major = [3, 5, 5, 3, 5, 5, 5]
harmonic_minor = [3, 5, 5, 3, 7, 3, 5]
melodic_mior = [3, 5, 5, 5, 5, 3, 5]
harmonic_major = [5, 3, 7, 3, 5, 5, 3]
```

In the above scales 1-step interval is forbidden and only major and minor 3rds are allowed.

In general we can also allow neutral, septimal minor, and septimal major thirds. We can also allow a 1-step interval but that sounds awful.

In [ ]:
def ji_to_edo(h, edo=31):
    return (edo * np.log2(h)) % edo

In [ ]:
ji_to_edo(7 / 3, 31)

In [ ]:
ji_to_edo(9 / 7, 31)

In [ ]:
ji_to_edo(11 / 9, 31)

In [ ]:
ji_to_edo(27 / 11, 31)

# TODO

- [X] A single function to generate all scales given allowed 3rds and allowed 2nds
- [ ] Play all modes of all scales using librosa or something.
- [X] Sort by number of perfect 5ths
- [ ] Find all scales where pitch classes = `[0, 5, *, 13, 18, *, *]` (why though?)

In [ ]:
def delta_to_pulse(deltas):
    pulses = sum([[1] + (k - 1) * [0] for k in deltas], start=[])
    return pulses


def pulse_to_delta(pulses):
    deltas = []
    delta = 1

    for idx, val in enumerate(pulses[1:]):
        if val == 0:
            delta += 1
            continue
        deltas.append(delta)
        delta = 1
    deltas.append(delta)

    return deltas